In [137]:
import pandas as pd
import pandas_gbq
from pandas.io import gbq
from datetime import date, timedelta, datetime
import ctypes
import chardet
import gcsfs
from io import StringIO
import google.auth
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import os
import win32com.client
import warnings
warnings.filterwarnings('ignore')
import time

# Credentials to extract data from the Bucket
fs = gcsfs.GCSFileSystem(project='nh-cro-forecast', token = r'C:\Users\ej.garcia\OneDrive - Minor Hotels Europe & Americas\Escritorio\nh-cro-forecast-ec5c044f54cf.json')

#DEFINIMOS AÑO, MES E INICIO DE DIA Y FINAL DE DIA
year = "2025"
print('Año: ',year)
mes = "01"
print("Mes: ",mes)

inicio_dia = "1"
print("Día inicio:", inicio_dia)

fin_dia = "17"
print("Día fin:", fin_dia)

Año:  2025
Mes:  01
Día inicio: 1
Día fin: 17


In [131]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+inicio_dia+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')

In [132]:
doc.head()

,grouplevel,Service,ENTRY_DATE,ENTRY_HOUR,ACTIVITY_ID,CASE_ID,Alias,AGENT_DISTRIBUTED,DISTRIBUTED_DATE,DISTRIBUTED_HOUR,...,due_on,from_mail,DIRECTION,REAL_TIME_ACTIVITY,MATCH,MATCH LEVEL,HOTEL,PAIS,CIUDAD,ID
0,0,Coordinacion,18/12/2024,16:07:26,126961953,110978798,a.dali@nh-hotels.com,SYSTEM SYSTEM,18/12/2024,16:39:17,...,20/12/2024 18:07:26,a.dali@minor-hotels.com,IN,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Coordinacion,18/12/2024,16:14:23,126962175,110975623,supportcro@nh-hotels.com,SYSTEM SYSTEM,18/12/2024,17:17:55,...,20/12/2024 18:14:23,j.van@minor-hotels.com,IN,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Coordinacion,18/12/2024,13:58:12,126958490,110977455,supportcro@nh-hotels.com,SYSTEM SYSTEM,18/12/2024,17:20:35,...,20/12/2024 15:58:12,reservierungen@nh-hotels.com,IN,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Coordinacion,18/12/2024,16:56:36,126963242,110979332,supportcro@nh-hotels.com,SYSTEM SYSTEM,18/12/2024,17:20:35,...,20/12/2024 18:56:36,leisurebookings@nh-hotels.com,IN,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Coordinacion,18/12/2024,18:21:57,126965114,110972213,supportcro@nh-hotels.com,SYSTEM SYSTEM,18/12/2024,19:35:45,...,20/12/2024 20:21:57,nhdiscovery@nh-hotels.com,IN,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
for i in range(int(inicio_dia),int(fin_dia)+1):
    if i <10: 
        Day = "0"+str(i)
        print(f"Subiendo datos del archivo de día:{Day}")
        with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
            doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
        doc = doc.iloc[:,1:]
        doc.columns=['Service',
                    'Entry_date',
                    'Entry_hour',
                    'Activity_ID',
                    'Case_ID',
                    'Alias',
                    'Agent_distributed',
                    'Distributed_date',
                    'Distributed_hour',
                    'Queue',
                    'Completed',
                    'Agent_completed',
                    'Completed_date',
                    'Completed_hour',
                    'Type',
                    'Subtype',
                    'due_on',
                    'From_mail',
                    'IN_OUT',
                    'Total_time',
                    'Match',
                    'Match_level',
                    'Hotel',
                    'Country',
                    'City',
                    'Hotel_Id'
                    ]
        # doc.insert(20,'Match','KKKKK')
        # doc.insert(25,'Hotel_ID','')
        raw = doc.astype(str)
        raw = raw.loc[:,['Service',
                'Entry_date',
                'Entry_hour',
                'Activity_ID',
                'Case_ID',
                'Alias',
                'Agent_distributed',
                'Distributed_date',
                'Distributed_hour',
                'Queue',
                'Completed',
                'Agent_completed',
                'Completed_date',
                'Completed_hour',
                'Type',
                'Subtype',
                'due_on',
                'From_mail',
                'IN_OUT',
                'Total_time',
                'Match',
                'Match_level',
                'Country',
                'City',
                'Hotel',
                'Hotel_Id']]
        raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
        raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
        time.sleep(0.5)

    else:
        Day = str(i)
        print(f"Subiendo datos del archivo de día:{Day}")
        with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
            doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
        doc = doc.iloc[:,1:]
        doc.columns=['Service',
                    'Entry_date',
                    'Entry_hour',
                    'Activity_ID',
                    'Case_ID',
                    'Alias',
                    'Agent_distributed',
                    'Distributed_date',
                    'Distributed_hour',
                    'Queue',
                    'Completed',
                    'Agent_completed',
                    'Completed_date',
                    'Completed_hour',
                    'Type',
                    'Subtype',
                    'due_on',
                    'From_mail',
                    'IN_OUT',
                    'Total_time',
                    'Match',
                    'Match_level',
                    'Hotel',
                    'Country',
                    'City',
                    'Hotel_Id'
                    ]
        # doc.insert(20,'Match','')
        # doc.insert(25,'Hotel_ID','')
        raw = doc.astype(str)
        raw = raw.loc[:,['Service',
                'Entry_date',
                'Entry_hour',
                'Activity_ID',
                'Case_ID',
                'Alias',
                'Agent_distributed',
                'Distributed_date',
                'Distributed_hour',
                'Queue',
                'Completed',
                'Agent_completed',
                'Completed_date',
                'Completed_hour',
                'Type',
                'Subtype',
                'due_on',
                'From_mail',
                'IN_OUT',
                'Total_time',
                'Match',
                'Match_level',
                'Country',
                'City',
                'Hotel',
                'Hotel_Id']]
        raw = doc.astype(str)
        raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
        raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
        time.sleep(0.5)

Subiendo datos del archivo de día:01


100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:02


100%|██████████| 1/1 [00:00<00:00, 1003.42it/s]


Subiendo datos del archivo de día:03


...

100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:16


100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:17


100%|██████████| 1/1 [00:00<?, ?it/s]


In [82]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
doc = doc.iloc[:,1:]

In [83]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11075 entries, 0 to 11074
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Service             11075 non-null  object 
 1   ENTRY_DATE          11075 non-null  object 
 2   ENTRY_HOUR          11075 non-null  object 
 3   ACTIVITY_ID         11075 non-null  int64  
 4   CASE_ID             11075 non-null  int64  
 5   Alias               6448 non-null   object 
 6   AGENT_DISTRIBUTED   11075 non-null  object 
 7   DISTRIBUTED_DATE    11075 non-null  object 
 8   DISTRIBUTED_HOUR    11075 non-null  object 
 9   QUEUE               11075 non-null  object 
 10  COMPLETED           11075 non-null  object 
 11  AGENT_COMPLETED     11075 non-null  object 
 12  COMPLETED_DATE      11075 non-null  object 
 13  COMPLETED_HOUR      11075 non-null  object 
 14  Type                11075 non-null  object 
 15  Subtype             11075 non-null  object 
 16  due_

In [44]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12927 entries, 0 to 12926
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Service            12927 non-null  object
 1   Entry_date         12927 non-null  object
 2   Entry_hour         12927 non-null  object
 3   Activity_ID        12927 non-null  object
 4   Case_ID            12927 non-null  object
 5   Alias              7496 non-null   object
 6   Agent_distributed  12927 non-null  object
 7   Distributed_date   12927 non-null  object
 8   Distributed_hour   12927 non-null  object
 9   Queue              12927 non-null  object
 10  Completed          12927 non-null  object
 11  Agent_completed    12927 non-null  object
 12  Completed_date     12927 non-null  object
 13  Completed_hour     12927 non-null  object
 14  Type               12927 non-null  object
 15  Subtype            12927 non-null  object
 16  due_on             12927 non-null  objec

In [41]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
doc = doc.iloc[:,1:]
doc.columns=['Service',
            'Entry_date',
            'Entry_hour',
            'Activity_ID',
            'Case_ID',
            'Alias',
            'Agent_distributed',
            'Distributed_date',
            'Distributed_hour',
            'Queue',
            'Completed',
            'Agent_completed',
            'Completed_date',
            'Completed_hour',
            'Type',
            'Subtype',
            'due_on',
            'From_mail',
            'IN_OUT',
            'Total_time',
            # 'Match',
            'Match_level',
            'Country',
            'City',
            'Hotel',
            # 'Hotel_ID'
            ]
doc.insert(20,'Match','KKKKK')
doc.insert(25,'Hotel_ID','KKKKK')


In [42]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12927 entries, 0 to 12926
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Service            12927 non-null  object
 1   Entry_date         12927 non-null  object
 2   Entry_hour         12927 non-null  object
 3   Activity_ID        12927 non-null  int64 
 4   Case_ID            12927 non-null  int64 
 5   Alias              7496 non-null   object
 6   Agent_distributed  12927 non-null  object
 7   Distributed_date   12927 non-null  object
 8   Distributed_hour   12927 non-null  object
 9   Queue              12927 non-null  object
 10  Completed          12927 non-null  object
 11  Agent_completed    12927 non-null  object
 12  Completed_date     12927 non-null  object
 13  Completed_hour     12927 non-null  object
 14  Type               12927 non-null  object
 15  Subtype            12927 non-null  object
 16  due_on             12927 non-null  objec